In [1]:
import pandas as pd

df=pd.read_csv("data/IMDB Dataset.csv")
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [4]:
print(df.review[0])

One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fac

In [5]:
classes = df['sentiment'].unique()
print(classes)

['positive' 'negative']


In [6]:
X,y= df['review'],df['sentiment']
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [8]:
import re
import html
# modified with GPT for best practices
def clean_text(text):
    text = html.unescape(text)
    text = text.lower()
    text = re.sub(r'<.*?>', ' ', text)             # remove HTML
    text = re.sub(r'http\S+|www\.\S+', ' ', text)  # remove URLs
    text = re.sub(r'\S+@\S+', ' ', text)           # remove emails
    text = re.sub(r"(.)\1{2,}", r"\1\1", text)     # soooo -> soo

    # keep ! and ? (sentiment)
    text = re.sub(r'[^a-z!? ]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text


Ref : https://www.kaggle.com/code/arunmohan003/sentiment-analysis-using-lstm-pytorch

and Copilot

In [ ]:
from nltk.corpus import stopwords
from gensim.parsing.preprocessing import preprocess_string
from collections import Counter
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

def tokenize(x_train, y_train, x_val, y_val, max_len=100):

    word_list = []
    stop_words = set(stopwords.words('english'))
    
    # Build vocabulary from training data
    print("Building vocabulary...")
    for sent in x_train:
        for word in sent.lower().split():
            processed = preprocess_string(word)
            if processed:  # Check if preprocess_string returned anything
                word = processed[0]
                if word not in stop_words and word != '':
                    word_list.append(word)
    
    # Create vocabulary dictionary
    corpus = Counter(word_list)
    corpus_ = sorted(corpus, key=corpus.get, reverse=True)[:1000]
    onehot_dict = {w: i+1 for i, w in enumerate(corpus_)}  # Start from 1, 0 is for padding
    
    print(f"Vocabulary size: {len(onehot_dict)}")
    
    # Tokenize sequences
    print("Tokenizing training data...")
    final_list_train = []
    for sent in x_train:
        tokens = []
        for word in sent.lower().split():
            processed = preprocess_string(word)
            if processed and processed[0] in onehot_dict:
                tokens.append(onehot_dict[processed[0]])
        final_list_train.append(tokens)
    
    print("Tokenizing validation data...")
    final_list_test = []
    for sent in x_val:
        tokens = []
        for word in sent.lower().split():
            processed = preprocess_string(word)
            if processed and processed[0] in onehot_dict:
                tokens.append(onehot_dict[processed[0]])
        final_list_test.append(tokens)
    
    # Pad sequences to same length
    print(f"Padding sequences to max length: {max_len}")
    padded_train = pad_sequences(final_list_train, maxlen=max_len, padding='post', truncating='post')
    padded_test = pad_sequences(final_list_test, maxlen=max_len, padding='post', truncating='post')
    
    # Encode labels
    encoded_train = np.array([1 if label == 'positive' else 0 for label in y_train])
    encoded_test = np.array([1 if label == 'positive' else 0 for label in y_val])
    
    print(f"Training data shape: {padded_train.shape}")
    print(f"Validation data shape: {padded_test.shape}")
    
    return padded_train, encoded_train, padded_test, encoded_test, onehot_dict

### GPT SAID
Perfect! The tokenizer is now working correctly. I've created a robust tokenizer that:

Properly handles preprocess_string() - Checks if it returns data and extracts the first element
Builds a vocabulary from the 1000 most common words in training data
Converts text to sequences of integer indices
Pads sequences to a uniform length (100 tokens) using pad_sequences from Keras
Encodes labels as binary values (1 for positive, 0 for negative)
Returns proper numpy arrays with consistent shapes
The output shows:

Vocabulary size: 1000 words
Training data shape: (40000, 100) - 40,000 samples, 100 tokens each
Validation data shape: (10000, 100) - 10,000 samples, 100 tokens each
You can now proceed with building your sentiment analysis model!

In [18]:
# import nltk
# nltk.download('stopwords')
# nltk.download('punkt')

In [21]:
x_train,y_train,x_test,y_test,vocab = tokenize(X_train,y_train,X_test,y_test)

Building vocabulary...
Vocabulary size: 1000
Tokenizing training data...
Tokenizing validation data...
Padding sequences to max length: 100
Training data shape: (40000, 100)
Validation data shape: (10000, 100)


In [ ]:
print(f'Length of vocabulary is {len(vocab)}')

In [22]:
def padding_(sentences, seq_len):
    features = np.zeros((len(sentences), seq_len),dtype=int)
    for ii, review in enumerate(sentences):
        if len(review) != 0:
            features[ii, -len(review):] = np.array(review)[:seq_len]
    return features

In [23]:
x_train_pad = padding_(x_train,500)
x_test_pad = padding_(x_test,500)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader
import torch

train_data = TensorDataset(torch.from_numpy(x_train_pad), torch.from_numpy(y_train))
valid_data = TensorDataset(torch.from_numpy(x_test_pad), torch.from_numpy(y_test))

batch_size = 50

train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size)

In [26]:
dataiter = iter(train_loader)
sample_x, sample_y = next(dataiter)

print('Sample input size: ', sample_x.size()) # batch_size, seq_length
print('Sample input: \n', sample_x)
print('Sample input: \n', sample_y)

Sample input size:  torch.Size([50, 500])
Sample input: 
 tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])
Sample input: 
 tensor([1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1,
        1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0,
        0, 1])


In [41]:
import torch.nn as nn

class SentimentLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers=2, bidirectional=False, dropout=.3): 
        super(SentimentLSTM, self).__init__()
        # input =(batch X seq_len)
        self.embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim, padding_idx=0) 
        # output = (batch X seq_len X embedding_dim) each seq token is represented by embedding_dim vector
        self.lstm = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim,
                           num_layers=num_layers, bidirectional=bidirectional, dropout=dropout, batch_first=True) 
        # output = (batch X seq_len X hidden_dim * num_directions) if bidirectional then num_directions=2 else 1
        self.dropout = nn.Dropout(dropout)
        # output = (batch X hidden_dim * num_directions) flattening
        self.fc = nn.Linear(in_features=hidden_dim*(2 if bidirectional else 1), out_features=1)
        # No sigmoid here - BCEWithLogitsLoss applies it internally
        
    def forward(self, x):
        embedded = self.embedding(x)  # (B X seq_len X embedding_dim)
        lstm_out, (hidden_state, cell_state) = self.lstm(embedded)  # lstm_out: (B X seq_len X H*num_directions)
        # hidden_state: (num_layers * num_directions, B, H)

        if self.lstm.bidirectional:
            h_last = torch.cat((hidden_state[-2], hidden_state[-1]), dim=1)  # (B, 2H)
        else:
            h_last = hidden_state[-1]  # (B, H)
            
        out = self.dropout(h_last)
        out = self.fc(out)  # (B, 1) - raw logits, no sigmoid
        return out

In [42]:
model=SentimentLSTM(vocab_size=len(vocab)+1,embedding_dim=128,hidden_dim=256,bidirectional=True,num_layers=2,dropout=0.3) # add 1 for padding index
print(model)

SentimentLSTM(
  (embedding): Embedding(1001, 128, padding_idx=0)
  (lstm): LSTM(128, 256, num_layers=2, batch_first=True, dropout=0.3, bidirectional=True)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=512, out_features=1, bias=True)
)


In [43]:
x = torch.randint(low=0, high=len(vocab), size=(32, 200))  # fake batch
with torch.no_grad():
    y = model(x)
print(y.shape)  # output= torch.Size([32, 1])

torch.Size([32, 1])


In [44]:
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
epochs = 5

In [ ]:
from tqdm import tqdm

for epoch in range(epochs):
    model.train()
    total_loss = 0
    
    for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
        inputs, labels = inputs.to(device), labels.to(device).float()
        
        optimizer.zero_grad()
        
        outputs = model(inputs).squeeze()  # squeeze to match labels shape
        
        loss = criterion(outputs, labels)  # calculate loss
        loss.backward()  # backpropagation
        optimizer.step()
        
        total_loss += loss.item()
    
    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{epochs}, Training Loss: {avg_loss:.4f}")
    
    # Validation
    model.eval()
    total_val_loss = 0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for inputs, labels in valid_loader:
            inputs, labels = inputs.to(device), labels.to(device).float()
            
            outputs = model(inputs).squeeze()  # raw logits
            
            val_loss = criterion(outputs, labels)
            total_val_loss += val_loss.item()
            
            # Apply sigmoid for prediction since model outputs raw logits
            predicted = (torch.sigmoid(outputs) >= 0.5).float()
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
            
    avg_val_loss = total_val_loss / len(valid_loader)
    val_accuracy = correct / total
    print(f"Epoch {epoch+1}/{epochs}, Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")
    print()

Epoch 1/5:   1%|          | 6/800 [06:19<13:38:55, 61.88s/it]

In [ ]:
def predict_sentiment(model, sentence, vocab, seq_len=500):
    model.eval()
    cleaned = clean_text(sentence)
    tokens = []
    for word in cleaned.split():
        processed = preprocess_string(word)
        if processed and processed[0] in vocab:
            tokens.append(vocab[processed[0]])
    padded_seq = padding_([tokens], seq_len)
    input_tensor = torch.from_numpy(padded_seq).to(device)
    
    with torch.no_grad():
        output = model(input_tensor).squeeze()
        prob = torch.sigmoid(output).item()
        
    sentiment = 'positive' if prob >= 0.5 else 'negative'
    return sentiment, prob

In [ ]:
print(predict_sentiment(model, "I really loved this movie! It was fantastic and thrilling.", vocab))